In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from transformers import TFAutoModel, AutoTokenizer

from tqdm.notebook import tqdm
from kaggle_datasets import KaggleDatasets

In [3]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [4]:
def regular_encode(texts, tokenizer, maxlen=512):
    # encode the word to vector of integer
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    return np.array(enc_di['input_ids'])

In [5]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
#     dense_layer = Dense(224, activation='relu')(cls_token)
    out = Dense(1, activation='sigmoid')(cls_token) # (dense_layer)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [6]:
AUTO = tf.data.experimental.AUTOTUNE

In [7]:
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 224

DATA_PATH = "/kaggle/input/jigsaw-multilingual-toxic-comment-classification/"
MODEL_DIR = '/kaggle/input/jplu-tf-xlm-roberta-large/'

In [8]:
tokenizer = AutoTokenizer.from_pretrained('jplu/tf-xlm-roberta-large')

In [9]:
train1 = pd.read_csv(os.path.join(DATA_PATH, "jigsaw-toxic-comment-train.csv"))
train2 = pd.read_csv(os.path.join(DATA_PATH, "jigsaw-unintended-bias-train.csv"))
train2.toxic = train2.toxic.round().astype(int)

In [10]:
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=2112)
])

In [11]:
valid = pd.read_csv(os.path.join(DATA_PATH, 'validation.csv'))
test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
sub = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

In [12]:
%%time

x_train = regular_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

CPU times: user 7min 31s, sys: 2.21 s, total: 7min 34s
Wall time: 7min 33s


In [13]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048, seed=2112)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [14]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL_DIR)
    model = build_model(transformer_layer, max_len=MAX_LEN)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 224)]             0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 224, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________
CPU times: user 32.2 s, sys: 29.4 s, total: 1min 1s
Wall time: 1min


In [15]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Train for 3404 steps, validate for 63 steps
Epoch 1/3


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


3404/3404 [==============================] - 1995s 586ms/step - loss: 0.0689 - accuracy: 0.9729 - val_loss: 0.3967 - val_accuracy: 0.8540
Epoch 2/3
3404/3404 [==============================] - 1789s 525ms/step - loss: 0.0534 - accuracy: 0.9785 - val_loss: 0.2992 - val_accuracy: 0.8614
Epoch 3/3
3404/3404 [==============================] - 1789s 526ms/step - loss: 0.0471 - accuracy: 0.9811 - val_loss: 0.2905 - val_accuracy: 0.8728


In [16]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

Train for 62 steps
Epoch 1/6


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


62/62 [==============================] - 71s 1s/step - loss: 0.2262 - accuracy: 0.8973
Epoch 2/6
62/62 [==============================] - 144s 2s/step - loss: 0.1586 - accuracy: 0.9286
Epoch 3/6
62/62 [==============================] - 32s 521ms/step - loss: 0.1067 - accuracy: 0.9545
Epoch 4/6
62/62 [==============================] - 32s 518ms/step - loss: 0.0768 - accuracy: 0.9701
Epoch 5/6
62/62 [==============================] - 32s 522ms/step - loss: 0.0435 - accuracy: 0.9845
Epoch 6/6
62/62 [==============================] - 32s 519ms/step - loss: 0.0345 - accuracy: 0.9872


In [17]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)

499/499 [==============================] - 122s 244ms/step
